In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time

plt.rcParams['text.usetex'] = True

# Functions

In [2]:
def demand_name_by_group_index(index):
    list_demand_names = ["Demand (eligible group, 1)", "Demand (eligible group, 2)", \
                         "Demand (ineligible group, 1)", "Demand (ineligible group, 2)", \
                         "Demand (ineligible group, 3)"]
    return list_demand_names[index]

def VoT_name_by_group_index(index):
    list_demand_names = ["VoT (eligible group, 1)", "VoT (eligible group, 2)", \
                         "VoT (ineligible group, 1)", "VoT (ineligible group, 2)", \
                         "VoT (ineligible group, 3)"]
    return list_demand_names[index]
    

In [3]:
# Function for transforming arrays of size (n, ) into arrays of size (n, 1)

def pad_dim(arr_or_list):
    arr = np.array(arr_or_list)
    assert len(arr.shape) == 1, "We must have len(arr.shape) == 1 to proceed"
    arr_len = arr.shape[0]
    return arr.reshape((arr_len, 1))

# Function for filling a vector from the bottom up to some value:

def fill_from_bottom(arr_or_list, val):
    arr = np.array(arr_or_list)
    assert np.all(arr >= -1E-3), "We must have all entries of arr >= 0.0."
    assert val >= -1E-3, "We must have val >= 0.0."
    
    arr_fill_from_bottom = np.zeros(arr.shape)
    
    index_boundary = max([index for index in range(arr.shape[0]) if np.sum(arr[index:]) >= val])
    
    for index in range(arr.shape[0]):
        if index > index_boundary:
            arr_fill_from_bottom[index] = arr[index]
        elif index < index_boundary:
            arr_fill_from_bottom[index] = 0.0
        else:
            arr_fill_from_bottom[index] = arr[index_boundary] - (np.sum(arr[index_boundary:]) - val)
        assert arr_fill_from_bottom[index] >= 0.0, "We must have arr_fill_from_bottom[index] >= 0.0"

    assert abs(np.sum(arr_fill_from_bottom) - val) <= 1E-3, \
        "We must have np.sum(arr_fill_from_bottom) == val"
    
    return arr_fill_from_bottom, index_boundary




In [4]:
# arr_temp = np.array([1, 2, 3])
# # len(arr_temp.shape)
# arr_temp_len = arr_temp.shape[0]
# arr_temp.reshape((arr_temp_len, 1))


# Groups, Routes to Edges:

In [5]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
# df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_1.csv')
df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_3.csv')

# df_od_flow_data
# df_data

In [6]:
dict_data = {}

for column_name_full in list(df_data.columns):
    if column_name_full == "Data Category":
        categories_list = df_data[column_name_full].tolist()
    else:
        dict_data[int(column_name_full)] = {}
        for category_index, category in enumerate(categories_list):
            if category == "Start City Index" or category == "End City Index":
                dict_data[int(column_name_full)][category] \
                    = int(df_data[column_name_full].tolist()[category_index])
            elif category == "Start City" or category == "End City":
                dict_data[int(column_name_full)][category] \
                    = df_data[column_name_full].tolist()[category_index]
            else:
#                 print("category:", category)
                dict_data[int(column_name_full)][category] \
                    = float(df_data[column_name_full].tolist()[category_index])

# Test git

In [7]:
dict_data

{0: {'Start City Index': 0,
  'End City Index': 0,
  'Start City': 'Palo Alto',
  'End City': 'Palo Alto',
  'O-D Flow (Max Entropy)': 612.5396169,
  'Demand (eligible group, 1)': 60.02888245620001,
  'VoT (eligible group, 1)': 0.04180157639979069,
  'Demand (eligible group, 2)': 36.752377014000004,
  'VoT (eligible group, 2)': 0.1494057158119658,
  'Demand (ineligible group, 1)': 93.71856138570001,
  'VoT (ineligible group, 1)': 0.29949880593262945,
  'Demand (ineligible group, 2)': 134.75871571800002,
  'VoT (ineligible group, 2)': 0.5809294871794872,
  'Demand (ineligible group, 3)': 287.28108032610004,
  'VoT (ineligible group, 3)': 1.8596449415012848},
 1: {'Start City Index': 0,
  'End City Index': 1,
  'Start City': 'Palo Alto',
  'End City': 'East Palo Alto',
  'O-D Flow (Max Entropy)': 98.03491986,
  'Demand (eligible group, 1)': 9.607422146280001,
  'VoT (eligible group, 1)': 0.04180157639979069,
  'Demand (eligible group, 2)': 5.8820951916,
  'VoT (eligible group, 2)': 0.149

In [8]:
cities_dict = {}
for od_info in list(dict_data.values()):
    if od_info["Start City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["Start City Index"]] = od_info["Start City"]
    if od_info["End City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["End City Index"]] = od_info["End City"]

cities_list = list(cities_dict.values())

# cities_dict

In [9]:
od_to_edges_array = np.zeros((len(list(dict_data.keys())), 2))

for od_index, od_info in dict_data.items():
    od_to_edges_array[od_index, 0] = int(cities_list.index(od_info["Start City"]))
    od_to_edges_array[od_index, 1] = int(cities_list.index(od_info["End City"]))

# od_to_edges_array

In [10]:
num_groups_per_od = 5

demand_array = np.zeros((len(list(dict_data.keys())), num_groups_per_od))
VoT_array_base = np.zeros((len(list(dict_data.keys())), num_groups_per_od))

for od_index, od_value in dict_data.items():
    for group_index in range(num_groups_per_od):
        demand_name = demand_name_by_group_index(group_index)
        VoT_name = VoT_name_by_group_index(group_index)
        
        demand_array[od_index, group_index] = od_value[demand_name]
        VoT_array_base[od_index, group_index] = od_value[VoT_name]

print(demand_array)
# VoT_array_base

[[ 60.02888246  36.75237701  93.71856139 134.75871572 287.28108033]
 [  9.60742215   5.88209519  14.99934274  21.56768237  45.97837741]
 [111.52479072  68.28048412 174.11523449 250.36177509 533.72578417]
 [ 25.07588823  15.35258463  39.14909081  56.29281031 120.00603653]
 [ 95.06622324  58.20381015 148.41971587 213.41397053 454.95978264]
 [ 23.76644387  14.550884    37.10475421  53.35324134 113.73940995]
 [150.04450532  91.86398285 234.25315626 336.83460377 718.07013259]
 [ 25.30172796  35.37078297  62.99613901  74.8724603   59.63978735]
 [  5.68890543   7.95285759  14.16421351  16.83451606  13.40956279]
 [ 21.56665392  30.14930191  53.6965669   63.81969017  50.83568424]
 [  5.39189555   7.5376499   13.42471953  15.95560927  12.70946808]
 [ 34.04016665  47.58676358  84.75306798 100.73110539  80.23753567]
 [ 10.40251464  13.91885761  23.5888008   37.36114412  61.2429735 ]
 [ 39.43629223  52.76686989  89.42595844 141.63738759 232.1742275 ]
 [  9.85945787  13.19223236  22.35736222  35.410

In [11]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'

T = 5
VoT_array = np.zeros((VoT_array_base.shape[0], VoT_array_base.shape[1], T))

for t in range(T):
    df_perturbation_data = pd.read_csv(directory_path + 'perturbations_1_' + str(t) + '.csv')
    perturbation_array = df_perturbation_data.to_numpy()[:, 1:]
    VoT_array[:, :, t] = VoT_array_base * perturbation_array
    
# VoT_array_base
# perturbation_array

# Generate sorted VoT and demand arrays

In [12]:
# arr_1 = np.array([[1, 2, 3], [4, 5, 6]])
# arr_2 = np.array([[11, 22, 33], [44, 55, 66]])
# arr = np.zeros((arr_1.shape[0], arr_1.shape[1], 2))
# arr[:, :, 0] = arr_1
# arr[:, :, 1] = arr_2

# arr_3 = np.array([[1, 2, 7], [4, 5, 6]])
# arr_3 = np.array([[13, 21, 7], [4, 45, 36]])
# np.any(arr_3 == arr_1)

arr_1 = np.array([1, 2, 3])
# pad_dim(arr_1)

# fill_from_bottom(arr_1, 4)


In [13]:
num_el = 3
num_groups = demand_array.shape[1]

# el_indices = [0, 1]
# in_indices = [2, 3, 4]

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

num_edges = int(np.max(od_to_edges_array)) + 1

edge_to_od_dict = {}
for e in range(num_edges):
    edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                           if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]



In [14]:
# e, t = 0, 0
# VoT_array_trunc = VoT_array[:, :, t][np.ix_(edge_to_od_dict[e], el_indices + in_indices)]
# VoT_array_trunc

In [15]:
# VoT_array_trunc == VoT_array[[0, 1, 2, 3, 4, 5, 6], :, t]

In [16]:
# e, t = 0, 0
# print("edge_to_od_dict[e]:", edge_to_od_dict[e])
# print("el_indices + in_indices:", el_indices + in_indices)

# # VoT_array_trunc = VoT_array[:, :, t][edge_to_od_dict[e], el_indices + in_indices]
# # VoT_array_trunc

In [17]:
## <font color='red'>Code edits start here (June 19)</font> 
## <font color='red'> (June 19) Comment out the following code and start over with more careful labeling of (od, g).</font> 

In [18]:
dict_VoTs_demands_annotated_sorted = {}

for e in range(num_edges):
    for t in range(T):
        
        print()
        print()
        print("e:", e)
        print("t:", t)
#         print("VoT_array_trunc.shape:", VoT_array_trunc.shape)
#         print("len(edge_to_od_dict[e]):", len(edge_to_od_dict[e]))

        VoT_array_trunc = VoT_array[:, :, t][np.ix_(edge_to_od_dict[e], el_indices + in_indices)]       
        demand_array_trunc = demand_array[np.ix_(edge_to_od_dict[e], el_indices + in_indices)]
        assert VoT_array_trunc.shape == demand_array_trunc.shape, \
            "We must have VoT_array_trunc.shape == demand_array_trunc.shape." 
        
        VoT_demand_array_trunc = np.zeros((VoT_array_trunc.shape[0], VoT_array_trunc.shape[1], 2))
        VoT_demand_array_trunc[:, :, 0] = VoT_array_trunc
        VoT_demand_array_trunc[:, :, 1] = demand_array_trunc
        
#         if e == 1 and t == 0:
#             print("VoT_array_trunc:\n", VoT_array_trunc)

        VoT_demand_in_annotated = np.zeros((VoT_array_trunc.shape[0] * len(in_indices), 4))
        VoT_demand_el_annotated = np.zeros((VoT_array_trunc.shape[0] * len(el_indices), 4))
        
        VoT_demand_in_arr_index = 0
        VoT_demand_el_arr_index = 0
        
        for row_index in range(VoT_array_trunc.shape[0]):
            for column_index in range(VoT_array_trunc.shape[1]):
                
                ## Ineligible users:                
                if column_index in in_indices:
                    
                    VoT_demand_in_annotated[VoT_demand_in_arr_index, 0] = VoT_array_trunc[row_index, column_index]
                    VoT_demand_in_annotated[VoT_demand_in_arr_index, 1] = demand_array_trunc[row_index, column_index]
                    VoT_demand_in_annotated[VoT_demand_in_arr_index, 2] = edge_to_od_dict[e][row_index]
                    VoT_demand_in_annotated[VoT_demand_in_arr_index, 3] = column_index
                    
                    VoT_demand_in_arr_index += 1

                ## Eligible users:
                elif column_index in el_indices:
                    
                    VoT_demand_el_annotated[VoT_demand_el_arr_index, 0] = VoT_array_trunc[row_index, column_index]
                    VoT_demand_el_annotated[VoT_demand_el_arr_index, 1] = demand_array_trunc[row_index, column_index]
                    VoT_demand_el_annotated[VoT_demand_el_arr_index, 2] = edge_to_od_dict[e][row_index]
                    VoT_demand_el_annotated[VoT_demand_el_arr_index, 3] = column_index
                    
                    VoT_demand_el_arr_index += 1
                else:
                    assert 1 == 0, "All column_index values should be in [0, 1, 2, 3, 4]. \
                        So, this case should not occur!"
        
        ## Sort VoTs and demands for both ineligible and eligible users:
        
        VoT_demand_in_annotated_sorted = VoT_demand_in_annotated[np.argsort(VoT_demand_in_annotated[:, 0])]
        VoT_demand_el_annotated_sorted = VoT_demand_el_annotated[np.argsort(VoT_demand_el_annotated[:, 0])]
        
        dict_VoTs_demands_annotated_sorted[(e, t, "in")] = VoT_demand_in_annotated_sorted
        dict_VoTs_demands_annotated_sorted[(e, t, "el")] = VoT_demand_el_annotated_sorted
        




e: 0
t: 0


e: 0
t: 1


e: 0
t: 2


e: 0
t: 3


e: 0
t: 4


e: 1
t: 0


e: 1
t: 1


e: 1
t: 2


e: 1
t: 3


e: 1
t: 4


e: 2
t: 0


e: 2
t: 1


e: 2
t: 2


e: 2
t: 3


e: 2
t: 4


e: 3
t: 0


e: 3
t: 1


e: 3
t: 2


e: 3
t: 3


e: 3
t: 4


e: 4
t: 0


e: 4
t: 1


e: 4
t: 2


e: 4
t: 3


e: 4
t: 4


e: 5
t: 0


e: 5
t: 1


e: 5
t: 2


e: 5
t: 3


e: 5
t: 4


e: 6
t: 0


e: 6
t: 1


e: 6
t: 2


e: 6
t: 3


e: 6
t: 4


In [19]:
edge_to_od_dict[1]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [20]:
dict_VoTs_demands_annotated_sorted

{(0,
  0,
  'in'): array([[4.87642507e-01, 5.62928103e+01, 3.00000000e+00, 3.00000000e+00],
        [5.38025558e-01, 2.15676824e+01, 1.00000000e+00, 3.00000000e+00],
        [5.58936305e-01, 2.13413971e+02, 4.00000000e+00, 3.00000000e+00],
        [5.60348837e-01, 5.33532413e+01, 5.00000000e+00, 3.00000000e+00],
        [5.65378293e-01, 2.50361775e+02, 2.00000000e+00, 3.00000000e+00],
        [5.89184313e-01, 1.34758716e+02, 0.00000000e+00, 3.00000000e+00],
        [6.95508618e-01, 3.36834604e+02, 6.00000000e+00, 3.00000000e+00],
        [1.50632339e+00, 4.54959783e+02, 4.00000000e+00, 4.00000000e+00],
        [1.60790372e+00, 1.13739410e+02, 5.00000000e+00, 4.00000000e+00],
        [1.73418502e+00, 2.87281080e+02, 0.00000000e+00, 4.00000000e+00],
        [1.77199060e+00, 5.33725784e+02, 2.00000000e+00, 4.00000000e+00],
        [2.04229666e+00, 1.20006037e+02, 3.00000000e+00, 4.00000000e+00],
        [2.09613598e+00, 4.59783774e+01, 1.00000000e+00, 4.00000000e+00],
        [2.13579654e

In [21]:
# dict_VoTs_demands_annotated = {}

# for e in range(num_edges):    
#     for t in range(T):
        
#         print()
#         print()
#         print("e:", e)
#         print("t:", t)
        
#         VoT_array_trunc = VoT_array[:, :, t][np.ix_(edge_to_od_dict[e], el_indices + in_indices)]
        
#         if e == 1 and t == 0:
#             print("VoT_array_trunc:\n", VoT_array_trunc)
        
#         demand_array_trunc = demand_array[np.ix_(edge_to_od_dict[e], el_indices + in_indices)]
#         VoT_demand_array_trunc = np.zeros((VoT_array_trunc.shape[0], VoT_array_trunc.shape[1], 2))
#         VoT_demand_array_trunc[:, :, 0] = VoT_array_trunc
#         VoT_demand_array_trunc[:, :, 1] = demand_array_trunc
        
#         ## Ineligible users:
                
#         VoT_array_trunc_in = VoT_array[:, :, t][np.ix_(edge_to_od_dict[e], in_indices)]
#         demand_array_trunc_in = demand_array[np.ix_(edge_to_od_dict[e], in_indices)]
#         assert np.abs(np.sum(demand_array_trunc_in) - sum([demand_array[od, g] for od in edge_to_od_dict[e] \
#                                                     for g in in_indices])) <= 1E-3
        
#         VoT_array_trunc_in_flattened = VoT_array_trunc_in.flatten()
#         demand_array_trunc_in_flattened = demand_array_trunc_in.flatten()
#         VoT_demand_in = np.block([pad_dim(VoT_array_trunc_in_flattened), pad_dim(demand_array_trunc_in_flattened)])   
#         VoT_demand_in_sorted = VoT_demand_in[np.argsort(VoT_demand_in[:, 0])]
        
# #         print("VoT_demand_in_sorted.shape:\n", VoT_demand_in_sorted.shape)
        
#         VoT_demand_in_annotations = np.zeros((VoT_demand_in_sorted.shape[0], 2))
#         VoT_demand_array_trunc_in = np.zeros((VoT_array_trunc_in.shape[0], VoT_array_trunc_in.shape[1], 2))
#         VoT_demand_array_trunc_in[:, :, 0] = VoT_array_trunc_in
#         VoT_demand_array_trunc_in[:, :, 1] = demand_array_trunc_in
# #         print("VoT_demand_array_trunc_in.shape:", VoT_demand_array_trunc_in.shape)

#         print("VoT_demand_in_annotations.shape:", VoT_demand_in_annotations.shape)
        
#         for index in range(VoT_demand_in_annotations.shape[0]):
# #             print()
            
#             od_g_indices_raw = [[index_0, index_1] for index_0 in range(VoT_demand_array_trunc.shape[0]) \
#                                                     for index_1 in range(VoT_demand_array_trunc.shape[1])
#                                 if np.all(VoT_demand_array_trunc[index_0, index_1, :] == VoT_demand_in_sorted[index])]
            
#             assert len(od_g_indices_raw) == 1, "We should have len(od_g_indices_raw) == 1, assuming no \
#                                                 duplicates in (VoT, demand) levels."
            
#             print("od_g_indices_raw:", od_g_indices_raw)
            
#             VoT_demand_in_annotations[index, 0] = od_g_indices_raw[0][0]
#             VoT_demand_in_annotations[index, 1] = od_g_indices_raw[0][1]

# #         print("VoT_demand_in_annotations:\n", VoT_demand_in_annotations)

#         VoT_demand_in_sorted_annotated = np.block([VoT_demand_in_sorted, VoT_demand_in_annotations])
        
#         dict_VoTs_demands_annotated[(e, t, "in")] = VoT_demand_in_sorted_annotated

# #         print("VoT_demand_in_annotations:\n", VoT_demand_in_annotations)
# #         print("VoT_demand_in_sorted_annotated:\n", VoT_demand_in_sorted_annotated)
        

#         ## Eligible users:

#         VoT_array_trunc_el = VoT_array[:, :, t][np.ix_(edge_to_od_dict[e], el_indices)]
#         demand_array_trunc_el = demand_array[np.ix_(edge_to_od_dict[e], el_indices)]

#         VoT_array_trunc_el_flattened = VoT_array_trunc_el.flatten()
#         demand_array_trunc_el_flattened = demand_array_trunc_el.flatten()
#         VoT_demand_el = np.block([pad_dim(VoT_array_trunc_el_flattened), pad_dim(demand_array_trunc_el_flattened)])   
#         VoT_demand_el_sorted = VoT_demand_el[np.argsort(VoT_demand_el[:, 0])]
        
#         VoT_demand_el_annotations = np.zeros((VoT_demand_el_sorted.shape[0], 2))
#         VoT_demand_array_trunc_el = np.zeros((VoT_array_trunc_el.shape[0], VoT_array_trunc_el.shape[1], 2))
#         VoT_demand_array_trunc_el[:, :, 0] = VoT_array_trunc_el
#         VoT_demand_array_trunc_el[:, :, 1] = demand_array_trunc_el
# #         print("VoT_demand_array_trunc_el.shape:", VoT_demand_array_trunc_el.shape)
        
#         for index in range(VoT_demand_el_annotations.shape[0]):
# #             print()
            
#             od_g_indices_raw = [[index_0, index_1] for index_0 in range(VoT_demand_array_trunc_el.shape[0]) \
#                                                     for index_1 in range(VoT_demand_array_trunc_el.shape[1])
#                                 if np.all(VoT_demand_array_trunc_el[index_0, index_1, :] == VoT_demand_el_sorted[index])]
            
#             assert len(od_g_indices_raw) == 1, "We should have len(od_g_indices_raw) == 1, assuming no \
#                                                 duplicates in (VoT, demand) levels."
            
#             VoT_demand_el_annotations[index, 0] = od_g_indices_raw[0][0]
#             VoT_demand_el_annotations[index, 1] = od_g_indices_raw[0][1]

#         VoT_demand_el_sorted_annotated = np.block([VoT_demand_el_sorted, VoT_demand_el_annotations])
        
# #         print("VoT_demand_el_annotations:\n", VoT_demand_el_annotations)    
# #         print("VoT_demand_el_sorted_annotated:\n", VoT_demand_el_sorted_annotated)

#         dict_VoTs_demands_annotated[(e, t, "el")] = VoT_demand_el_sorted_annotated
        

In [22]:
# dict_VoTs_demands_annotated

# Save to CSV

In [23]:

# column_names = ["VoT", "demand", "od", "g"]
# df_in_save = pd.DataFrame(dict_VoTs_demands_annotated_sorted[(0, 0, "in")], columns=column_names)

In [24]:
directory_path = "../data/VoTs_demands_sorted___" + str(num_el) + "_el_groups/"

column_names = ["VoT", "demand", "od", "g"]

for e in range(num_edges):
    for t in range(T):
#         print()
        
        # Ineligible users:
        
        df_in_save = pd.DataFrame(dict_VoTs_demands_annotated_sorted[(e, t, "in")], columns = column_names)
        df_in_save["od"] = df_in_save["od"].astype(int)
        df_in_save["g"] = df_in_save["g"].astype(int)
        
        filename_in = str(e) + "_" + str(t) + "_" + "in.csv"
        df_in_save.to_csv(directory_path + filename_in, index=False)
        
        # Eligible users:
        
        df_el_save = pd.DataFrame(dict_VoTs_demands_annotated_sorted[(e, t, "el")], columns = column_names)
        df_el_save["od"] = df_el_save["od"].astype(int)
        df_el_save["g"] = df_el_save["g"].astype(int)
        
        filename_el = str(e) + "_" + str(t) + "_" + "el.csv"
        df_el_save.to_csv(directory_path + filename_el, index=False)
        

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()
